# Data Cleaning & Exploration - AI JOB MARKET EXPLORATION 
In this notebook, I will be merging the two Kaggle datasets into one 

In [2]:
import pandas as pd
import numpy as np
import os

# Loading Raw Datasets
Will read both raw datasets

In [3]:
df1 = pd.read_csv("../data/raw/ai_job_dataset.csv")
df2 = pd.read_csv("../data/raw/ai_job_dataset1.csv")

print("Dataset 1:", df1.shape)
print("Dataset 2:", df2.shape)


Dataset 1: (15000, 19)
Dataset 2: (15000, 20)


# Merging the two
Will be merging the two datasets

In [ ]:
merged_df = pd.merge(df1, df2, on='job_id', how='inner')
print("Merged dataset shape:", merged_df.shape)

Merged dataset shape: (15000, 38)


# Understanding and Exlploring Data
Will look into dataset

In [13]:
merged_df.info()
merged_df.describe()
merged_df.columns
merged_df.nunique()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 38 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   job_id                    15000 non-null  object 
 1   job_title_x               15000 non-null  object 
 2   salary_usd_x              15000 non-null  int64  
 3   salary_currency_x         15000 non-null  object 
 4   experience_level_x        15000 non-null  object 
 5   employment_type_x         15000 non-null  object 
 6   company_location_x        15000 non-null  object 
 7   company_size_x            15000 non-null  object 
 8   employee_residence_x      15000 non-null  object 
 9   remote_ratio_x            15000 non-null  int64  
 10  required_skills_x         15000 non-null  object 
 11  education_required_x      15000 non-null  object 
 12  years_experience_x        15000 non-null  int64  
 13  industry_x                15000 non-null  object 
 14  postin

job_id                      15000
job_title_x                    20
salary_usd_x                14315
salary_currency_x               3
experience_level_x              4
employment_type_x               4
company_location_x             20
company_size_x                  3
employee_residence_x           20
remote_ratio_x                  3
required_skills_x           13663
education_required_x            4
years_experience_x             20
industry_x                     15
posting_date_x                486
application_deadline_x        543
job_description_length_x     2000
benefits_score_x               51
company_name_x                 16
job_title_y                    20
salary_usd_y                14359
salary_currency_y               8
salary_local                14458
experience_level_y              4
employment_type_y               4
company_location_y             20
company_size_y                  3
employee_residence_y           50
remote_ratio_y                  3
required_skill

In [14]:
merged_df.isnull().sum().sort_values(ascending=False)

job_id                      0
remote_ratio_y              0
salary_currency_y           0
salary_local                0
experience_level_y          0
employment_type_y           0
company_location_y          0
company_size_y              0
employee_residence_y        0
required_skills_y           0
job_title_x                 0
education_required_y        0
years_experience_y          0
industry_y                  0
posting_date_y              0
application_deadline_y      0
job_description_length_y    0
benefits_score_y            0
salary_usd_y                0
job_title_y                 0
company_name_x              0
benefits_score_x            0
salary_usd_x                0
salary_currency_x           0
experience_level_x          0
employment_type_x           0
company_location_x          0
company_size_x              0
employee_residence_x        0
remote_ratio_x              0
required_skills_x           0
education_required_x        0
years_experience_x          0
industry_x

# Cleaning & Organizing Dataset as to how I want it
Will be dropping Salary_Local Column
Will be moving the _y data vertically to essentially have 300000 different job_id
Will be standardizing the data (no mor _x and _y labels)

In [22]:
# 1. Drop salary_local
merged_df = merged_df.drop(columns=['salary_local'], errors='ignore')

# 2. Split _x and _y datasets (plus job_id for both)
x_cols = ['job_id'] + [col for col in merged_df.columns if col.endswith('_x')]
y_cols = ['job_id'] + [col for col in merged_df.columns if col.endswith('_y')]

df_x = merged_df[x_cols].copy()
df_y = merged_df[y_cols].copy()

# 3. Rename columns: remove _x and _y suffixes
df_x.columns = ['job_id'] + [col.replace('_x', '') for col in df_x.columns if col != 'job_id']
df_y.columns = ['job_id'] + [col.replace('_y', '') for col in df_y.columns if col != 'job_id']

# 4. Stack vertically (ignore old job_id for now)
df_combined = pd.concat([df_x, df_y], axis=0).reset_index(drop=True)

# 5. Reassign job_ids from AI00001 to AI30000
df_combined['job_id'] = ['AI' + str(i).zfill(5) for i in range(1, len(df_combined) + 1)]

df_combined[df_combined['job_id'] == 'AI15001']




,job_id,job_title,salary_usd,salary_currency,experience_level,employment_type,company_location,company_size,employee_residence,remote_ratio,required_skills,education_required,years_experience,industry,posting_date,application_deadline,job_description_length,benefits_score,company_name
15000,AI15001,Data Scientist,219728,USD,EX,PT,Sweden,M,Sweden,0,"Python, Computer Vision, R, Docker",Associate,13,Transportation,2024-09-23,2024-10-31,1132,6.6,TechCorp Inc


# Save the combined data set


In [23]:
# Save to processed CSV
output_path = '../data/processed/ai_job_dataset_combined.csv'
df_combined.to_csv(output_path, index=False)

print(f"Saved combined dataset to: {output_path}")


Saved combined dataset to: ../data/processed/ai_job_dataset_combined.csv
